In [2]:
import cv2
import os
import sys
import re
from keras.preprocessing import image
from keras.models import load_model
import numpy as np
import tkinter as tk
from tkinter import ttk
import tkinter.font as TkFont
from tkinter import filedialog
import platform
from PIL import Image, ImageTk

In [3]:
gui_img_global = None
resize_image = None
filename = None
img_input_canvas = None
img_ratio = None
img_prediction_canvas = None
classes = []

def build_gui():
    root = tk.Tk()

    root.title("Human Identification System based on Ear Shape using Convolutional Neural Network")

    if platform.system() == "Windows":
        root.iconbitmap(r'Assets/ear.ico')
    elif platform.system() == "Darwin":
        img = tk.Image("photo", file = r'Assets/ear.gif')
        root.iconphoto(True, img)
    
    root.geometry('900x600')
    root.resizable(width = False, height = False)

    header = TkFont.Font(family = "San Francisco", size = 24, weight = "bold")
    subheader = TkFont.Font(family = "San Francisco", size = 10)

    base_canvas = tk.Canvas(root, width = 900, height = 600)
    base_canvas.pack()
    
    title = tk.Label(text = "Human Identification System based on\nEar Shape using Convolutional Neural Network", font = header)
    base_canvas.create_window(450, 60, window = title)

    def def_orientation(image):
        height = image.shape[0]
        width = image.shape[1]

        if height > width:
            orientation = "portrait"
        else:
            orientation = "landscape"
        return orientation

    def resize_image_gui(image, orientation, height, width):
        img_ratio = 1
        
        if (orientation == "portrait"):
            if height > 300:
                img_ratio = 300 / height
            image = image.resize((int(img_ratio * width), int(img_ratio * height)))
        else:
            if width > 272:
                img_ratio = 272 / width
            image = image.resize((int(img_ratio * width), int(img_ratio * height)))
        return image

    def upload_file():
        global img_input_canvas
        global img_ratio
        global gui_img_global
        global resize_image
        global filename
        global classes

        if img_input_canvas:
            base_canvas.delete(img_input_canvas)
            img_ratio = 1
        
        file_types = [('JPG Files', '*.jpg'),
                      ('JPEG Files', '*.jpeg'),
                      ('PNG Files', '*.png')
                    ]
        
        filename = filedialog.askopenfilename(title = 'Select file', filetypes = file_types)
        gui_img_global = cv2.imread(filename)
        orientation = def_orientation(gui_img_global)

        if filename:
            filepath = os.path.abspath(filename)      
            
        image = Image.open(filepath)
        resize_image = resize_image_gui(image, orientation, gui_img_global.shape[0], gui_img_global.shape[1])
        display = ImageTk.PhotoImage(resize_image)

        label1 = tk.Label(image=display)
        label1.image = display
        img_input_canvas = base_canvas.create_window(225, 365, window = label1)

        make_prediction_label2 = tk.Label(text = "                                         ", font = header)
        base_canvas.create_window(675, 365, window = make_prediction_label2)
        
        make_prediction_button['state'] = tk.NORMAL
        
    def load_classes():
        data_dir = 'EarVN1.0 dataset/Images'
        
        for idx, folder_name in enumerate(os.listdir(data_dir)):
            classes.append(folder_name)
            
        classes.sort()
        
    def clean_result(name):
        extracted_name = re.split(r'[.]', name)
        replaced_underscore_name = re.sub(r'[_]', ' ', extracted_name[1])
        return replaced_underscore_name
        
    def make_prediction():
        classes.clear()
        load_classes()
        
        img_width, img_height = 224, 224
        img = image.load_img(filename, target_size = (img_width, img_height))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        
        model_path = 'EfficientNetV2-S 100 epoch/best_efficientnetv2s_model_epoch=96_acc=0.9158.h5'
        model = load_model(model_path)
        
        result = model.predict(img)
        
        if platform.system() == "Windows":
            name = classes[np.argmax(result)]
        elif platform.system() == "Darwin":
            name = classes[np.argmax(result) + 1]
        
        final_result = clean_result(name)
        
        make_prediction_label2 = tk.Label(text = "                                         ", font = header)
        base_canvas.create_window(675, 365, window = make_prediction_label2)
        
        make_prediction_label = tk.Label(text = final_result, font = header)
        base_canvas.create_window(675, 365, window = make_prediction_label)

    upload_image_frame = tk.Frame(root, width = 345, height = 425, highlightbackground = "grey", highlightthickness= 1)
    upload_image_frame.place(x = 55, y = 125)
    
    make_prediction_frame = tk.Frame(root, width = 345, height = 425, highlightbackground = "grey", highlightthickness= 1)
    make_prediction_frame.place(x = 500, y = 125)
    
    upload_image_button = tk.Button(root, text = "Upload Image", font = subheader, command = upload_file, fg = "black", bg = "white")
    base_canvas.create_window(225, 175, window = upload_image_button)
    
    make_prediction_button = tk.Button(root, text = "Make Prediction", font = subheader, command = make_prediction, fg = "black", bg = "white", state = tk.DISABLED)
    base_canvas.create_window(675, 175, window = make_prediction_button)

    root.mainloop()

In [4]:
if __name__ == '__main__':
    build_gui()

1/1 [==============================] - 1s 898ms/step
